<a href="https://colab.research.google.com/github/jtlagumbay/cebqa/blob/main/retriever/bm_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **QA Pipeline**

1. ElasticSeach Indexer
2. BM25 Retriever
3. Fine-tuned XLMR Reader


# Dependencies

In [6]:
pip install elasticsearch transformers datasets evaluate rank_bm25 nltk fuzzywuzzy sentence_transformers numpy==1.26.4


  Using cached numpy-1.26.4-cp310-cp310-macosx_10_9_x86_64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 3.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install --upgrade --no-cache-dir numpy==1.26.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 44.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 3.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import json
import os
import subprocess
import time
import pandas as pd
import requests
from transformers import pipeline, AutoModelForQuestionAnswering, AutoTokenizer
from datasets import Dataset, load_dataset
import re
from evaluate import load
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from fuzzywuzzy import fuzz
import nltk
import random
import numpy as np
from sentence_transformers import SentenceTransformer, util
import torch
from nltk.tokenize import sent_tokenize


nltk.download('punkt')
nltk.download('punkt_tab')

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jhoannaricalagumbay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/jhoannaricalagumbay/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
CEBQA_DATASET = "jhoannarica/cebquad"
# ELASTIC_URL = "https://tender-separately-mudfish.ngrok-free.app"
# CURRENT_MODEL = "/content/drive/MyDrive/UP Files/IV - 2nd sem/CMSC 198.1/cebqa_roberta/xlmr/2025-03-15_09-36/model"
# CURRENT_TOKENIZER = "/content/drive/MyDrive/UP Files/IV - 2nd sem/CMSC 198.1/cebqa_roberta/xlmr/2025-03-15_09-36/tokenizer"
CURRENT_MODEL = "/Users/jhoannaricalagumbay/Library/CloudStorage/GoogleDrive-jtlagumbay@up.edu.ph/My Drive/UP Files/IV - 2nd sem/CMSC 198.1/cebqa_roberta/xlmr/2025-03-15_09-36/model"
CURRENT_TOKENIZER = "/Users/jhoannaricalagumbay/Library/CloudStorage/GoogleDrive-jtlagumbay@up.edu.ph/My Drive/UP Files/IV - 2nd sem/CMSC 198.1/cebqa_roberta/xlmr/2025-03-15_09-36/tokenizer"
ELASTIC_URL = "http://localhost:9200"
INDEX_NAME = "superbalita"
K = 3

# Indexer

Start ElasticSearch Locally:
1. Start ES docker
2. Start NGROK: `ngrok http --url=tender-separately-mudfish.ngrok-free.app 9200`


In [3]:
headers = {
    "Origin": "https://colab.research.google.com",
     "Content-Type": "application/json",
}

response = requests.options(ELASTIC_URL, headers=headers)
print(response.headers)


{'Allow': 'GET,DELETE,HEAD', 'X-elastic-product': 'Elasticsearch', 'content-type': 'text/plain; charset=UTF-8', 'content-length': '0'}


In [4]:
es = Elasticsearch([ELASTIC_URL], verify_certs=False, headers=headers)
print(es.info())
# try:
#     print(es.transport.perform_request('GET', '/'))
# except Exception as e:
#     print("Error:", e)

{'name': '40645cf85c9d', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'ST_2iQqITE-zmJCo8dlrnw', 'version': {'number': '8.17.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '747663ddda3421467150de0e4301e8d4bc636b0c', 'build_date': '2025-02-05T22:10:57.067596412Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [5]:
class ElasticSearchIndexer:
    def __init__(self, index_name=INDEX_NAME):
        self.index_name = index_name
        self.es = Elasticsearch(ELASTIC_URL)  # Ensure ES is running

    def create_index(self):
        """ Create an index with a text field for BM25 """
        if not self.es.indices.exists(index=self.index_name):
            self.es.indices.create(index=self.index_name, body={
                "settings": {
                    "number_of_shards": 1,
                    "number_of_replicas": 0
                },
                "mappings": {
                    "properties": {
                        "id": {"type": "keyword"},
                        "title": {"type": "text"},
                        "body": {"type": "text"}
                    }
                }
            })
            print(f"Index '{self.index_name}' created.")

    def index_documents(self, documents):
        """ Bulk index documents into ElasticSearch """
        actions = [
            {
                "_index": self.index_name,
                "_id": doc["id"],  # Use document ID for uniqueness
                "_source": {
                    "id": doc["id"],
                    "title": doc["pseudonymized_title"],
                    "body": doc["pseudonymized_body"]
                }
            }
            for doc in documents
        ]
        bulk(self.es, actions)
        print(f"Indexed {len(documents)} documents.")

    def index_from_csv(self, file_path):
        df = pd.read_csv(file_path)
        documents = df.to_dict(orient="records")  # Convert DataFrame to a list of dicts
        self.index_documents(documents)



In [ ]:
# Sample usage
# indexer = ElasticSearchIndexer()
# indexer.create_index()
# indexer.index_from_csv("/content/drive/MyDrive/UP Files/IV - 2nd sem/CMSC 198.1/cebqa_roberta/articles_dataset.csv")

# BM25

In [66]:
class BM25Retriever:
    def __init__(self, index_name="superbalita"):
        self.index_name = index_name
        self.es = Elasticsearch(ELASTIC_URL)

    def retrieve(self, query, top_k=3):
        """ Retrieve top-k relevant documents using BM25 """
        print(f"retrieving {top_k} docs for [{query}]")
        response = self.es.search(index=self.index_name, body={
            "query": {
                "match": {
                    "body": query
                }
            },
            "size": top_k
        })
        return [hit["_source"] for hit in response["hits"]["hits"]]

    def retrieve_batch(self, queries, top_k=3):
        """ Retrieve top-k relevant documents for multiple queries using BM25 in batch mode """
        if not isinstance(queries, list):
            raise ValueError("queries should be a list of strings")

        # Multi-search request body
        request_body = ""
        for query in queries:
            request_body += f'{{"index": "{self.index_name}"}}\n'  # Metadata
            request_body += f'{{"query": {{"match": {{"body": "{query}"}}}}, "size": {top_k}}}\n'  # Query

        # Send multi-search request
        response = self.es.msearch(body=request_body)

        # Extract results
        results = []
        for query_response in response["responses"]:
            retrieved_docs = [hit["_source"] for hit in query_response["hits"]["hits"]]
            results.append(retrieved_docs)

        return results  # List of lists, where each sublist contains retrieved documents for a query

    def retrieve_batch_query_dict(self, queries_list, top_k=3):
        """ Retrieve top-k relevant documents for multiple queries using BM25 in batch mode.

        Args:
            queries_list (list): A list of dictionaries, each containing 'id' and 'question'.
            top_k (int): Number of top relevant documents to retrieve per query.

        Returns:
            dict: A dictionary where keys are query IDs and values are lists of retrieved documents.
        """
        if not isinstance(queries_list, list) or not all(isinstance(q, dict) and 'id' in q and 'question' in q for q in queries_list):
            raise ValueError("queries_list should be a list of dictionaries with 'id' and 'question' keys")

        # Multi-search request body
        request_body = ""
        query_ids = []  # To track IDs in order
        for query in queries_list:
            query_ids.append(query["id"])
            request_body += f'{{"index": "{self.index_name}"}}\n'  # Metadata
            request_body += f'{{"query": {{"match": {{"body": "{query["question"]}"}}}}, "size": {top_k}}}\n'  # Query

        # Send multi-search request
        response = self.es.msearch(body=request_body)

        # Extract results and associate with query IDs
        results = []
        for i, query_response in enumerate(response["responses"]):
            retrieved_docs = [hit["_source"] for hit in query_response["hits"]["hits"]]
            results.append({
                "query_id": str(query_ids[i]),
                "top_docs": retrieved_docs
            })

        return results  # Dictionary format: {id: retrieved_docs}



In [67]:
# Sample usage
retriever = BM25Retriever()
query = "kanus-a ang palarong pambansa"
top_docs = retriever.retrieve(query)
print("Retrieved Documents:", top_docs)

retrieving 3 docs for [kanus-a ang palarong pambansa]
Retrieved Documents: [{'id': 535, 'title': 'Palaro budget gikunhoran sa P262M gikan sa P405M', 'body': 'Gitugotan ni Cebu City Acting Mayor Jhonnylou Ghen Abalayan ang pagkunhod ngadto sa P262 milyunes gikan sa P405 milyunes ang gahin alang sa Palarong Pambansa 2024 sa Hulyo. Sa press conference sa Lunes, Hunyo 3, 2024, gibutyag ni Abalayan nga dugang budget appropriations alang sa pag-host sa Palarong Pambansa ang giputol, nga naglakip sa extension sa grandstand nga gitangtang na. “I have authorized only the charging of P262 million out of the P405 million. There is a cut of P143 million,” matod ni Abalayan. Si Abalayan niingon nga ang budget alang sa pagtukod sa grandstand extension mahimo pa nga maapil sa budget appropriations sa sunod tuig. Si Abalayan niingon usab nga iyang gius-usan ang budget alang sa side events gikan sa P60 milyunes ngadto sa P20 milyunes. Siya nidugang nga ang budget alang sa LED walls nga P135 milyunes gi

# Reader

In [ ]:
class Reader:
    def __init__(
        self,
        model_path = CURRENT_MODEL,
        tokenizer_path = CURRENT_TOKENIZER
      ):
        model_best = AutoModelForQuestionAnswering.from_pretrained(model_path)
        tokenizer_best = AutoTokenizer.from_pretrained(tokenizer_path)

        device = torch.device("mps")
        self.qa_pipeline = pipeline(
            "question-answering", 
            model=model_best, 
            tokenizer=tokenizer_best,
            device=device
            )

    def extract_answer_batch(self, queries_list, top_docs):
        qa_dataset = Dataset.from_dict({
          "question": [queries_list["question"] for doc in top_docs['top_docs']] ,
          "context": [doc['body'] for doc in top_docs['top_docs']]
        })

        return self.qa_pipeline(qa_dataset)

    def extract_answer(self, question, documents, num_chunks = 1, overlap = 0.3):
        """ Find the best answer from retrieved documents while keeping metadata """
        best_result = None
        best_score = 0

        for doc in documents:
            if num_chunks == 1:
                contexts = [doc["body"]]
            else:
                contexts = self.chunk_text(doc["body"],  num_chunks, overlap)

            for context in contexts:
            #   print(question)
            #   print(context)
              result = self.qa_pipeline(question=question, context = context)
              if result["score"] > best_score:
                  best_result = {
                      "article_id": doc["id"],
                      "title": doc["title"],
                      "body": doc["body"],
                      "answer": result["answer"],
                      "score": result["score"]
                  }
                  best_score = result["score"]

        return best_result

    def chunk_text(self, text, chunk_size=3, overlap=0.5):
        sentences = sent_tokenize(text)  # Tokenize text into sentences
        step = int(chunk_size * (1 - overlap))  # Overlapping step

        chunks = []
        for i in range(0, len(sentences), step):
            chunk = sentences[i:i + chunk_size]
            if not chunk: 
                continue
            chunks.append(" ".join(chunk)) 

        return chunks

# QA Pipeline

In [119]:
class QA:
    def __init__(
        self,
        model_path = CURRENT_MODEL,
        tokenizer_path = CURRENT_TOKENIZER,
        dataset = CEBQA_DATASET,
        index_name = INDEX_NAME,
        elastic_url = ELASTIC_URL,
        k = K,
        sample = None,
        isRandom = False,
        overlap = 0.0, 
        num_chunks = 1
      ):
        reader = Reader(model_path=model_path, tokenizer_path=tokenizer_path)
        retriever = BM25Retriever(index_name=index_name)
        
        self.model_path = model_path
        self.tokenizer_path = tokenizer_path
        self.reader = reader
        self.retriever = retriever
        self.dataset = load_dataset(dataset)["test"]
        self.sentence_transformer = SentenceTransformer("all-MiniLM-L6-v2")
        self.k = k
        self.overlap = overlap
        self.num_chunks = num_chunks
        self.sample = sample
        self.isRandom = isRandom
        self.index_name = index_name

        if sample is not None and isRandom:
            indices = random.sample(range(len(self.dataset)), sample)
            self.dataset = self.dataset.select(indices)
        elif sample is not None and not isRandom:
            self.dataset = self.dataset.select(range(sample))

        self.run_top_docs_batch()

    def run_top_docs_batch(self):
        queries = [
            {
                "id": item['id'],
                "question": item['question'],
                "context": {
                    "text": item['context']['text'],
                    "start": item['context']['start']
                }
             }
            for item in self.dataset
          ]
        self.top_docs = self.retriever.retrieve_batch_query_dict(
            queries_list = queries,
            top_k=self.k
        )

        return self.top_docs

    def run(self):
        start_time = time.time()
        queries = [
            {
                "id": item['id'],
                "question": item['question'],
                "context": {
                    "text": item['context']['text'],
                    "start": item['context']['start']
                },
                "answer": item['answer']
             }
            for item in self.dataset
          ]
        

        results = []

        for index, query in enumerate(queries):
          print(f"{index} processing {query['id']}")
          docs = self.top_docs[index]['top_docs']
        #   print([f"{doc}\n" for doc in docs])
          answer = self.reader.extract_answer(
              question = query["question"],
              documents = docs,
              num_chunks = self.num_chunks,
              overlap= self.overlap
          )
        #   print(query["answer"]["text"])
        #   print(answer["answer"])
          result = query
          result["pred"] = answer
          result["top_docs"] = docs
          results.append(result)

        self.results = results

        end_time = time.time()
        self.stats ={
            'run_time': end_time - start_time
        }
        return self.results


    def normalize_text(self, text):
        """Lowercase and remove punctuation, articles, and extra whitespace."""
        text = text.lower()
        text = re.sub(r'\W+', ' ', text)  # Remove punctuation and special characters
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
        return text

    def compute_similarity(self, text1, text2):
        """Compute cosine similarity between two texts using Sentence Transformers."""
        emb1 = self.sentence_transformer.encode(text1, convert_to_tensor=True)
        emb2 = self.sentence_transformer.encode(text2, convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(emb1, emb2).item()  # Convert tensor to float
        return similarity

    def evaluate_batch(self):
        pass

    def evaluate(self):
        pred = [
          {
              'id': result['id'],  # Convert ID to string
              'prediction_text': self.normalize_text(result['pred']['answer'])
          }
          for result in self.results
        ]

        ref = [
            {
                'id': item['id'],  # Convert ID to string
                'answers': {
                    'text': [self.normalize_text(item['answer']['text'])],
                    'answer_start': [item['answer']['start']]
                }
            }
            for item in self.results
        ]

        # Load SQuAD metric
        metric = load("squad")

        # Compute metric
        res = metric.compute(predictions=pred, references=ref)
        sentence_match_scores = [
            p['prediction_text'] in r['answers']['text'][0] for p, r in zip(pred, ref)
        ]

        # Compute average sentence match score
        avg_sentence_match = np.mean(sentence_match_scores)

        # Combine results
        res["sentence_match"] = float(avg_sentence_match )
        print(res)

        self.config = {
            'model_path': self.model_path,
            'tokenizer_path': self.tokenizer_path,
            'k': self.k,
            'sample': self.sample,
            'isRandom': self.isRandom,
            'overlap': self.overlap,
            'num_chunks': self.num_chunks
        }
        self.eval_res = res

        return self.eval_res, self.config, self.stats

In [12]:
print(np.__version__)  # Check if NumPy is available
print(torch.randn(1).numpy()) 


1.26.4
[-0.18783537]


In [120]:
qa = QA(sample=5, k = 1, overlap=0.0, num_chunks=2, isRandom=True)

In [121]:
qa.run()

0 processing 00200-018
['Usa ka drug den nga nahimutang sa Sitio Tabay Lawum, Barangay Tisa, Siyudad sa Sugbo ang gibungkag sa mga sakop sa Philippine Drug Enforcement Agency (PDEA) 7 uban sa Regional Intelligence Unit sa Police Regional Office (PRO) 7, alas 6:25 sa hapon, Sabado, Agusto 24, 2024, nga niresulta sa pagkasikop sa tulo ka mga tawo. Nadakpan sa buy-bust ang ilang target nga mao usab ang nagpadagan sa balay suyopanan nga giila nga sila si Nolan Bigcas alyas Joshua, 42, nagmintinar sa suyopanan; Enan Biglang, 43, masahista; ug Junrex Siaboc Dalia, 38, drayber sa habalhabal.', 'Nasakmit sa mga sakop sa PDEA 7 ug PRO 7 ang 18 ka pakete sa gituohang shabu nga motimbang sa 16 gramos nga dunay estimated average market value P108,800 ug ang mga drug paraphernalia. Ang maong mga ebidensya gi-turnover na sa buhatan sa PDEA 7 alang sa himuon nga chemical analysis.', 'Matod ni Ason Agapay, information officer sa PDEA 7, usa ka buwan nila nga gipaubos sa case buildup si Bigcas human ni

[{'id': '00200-018',
  'question': 'Unsang trabaho ang gihupot ni Enan Biglang?',
  'context': {'text': 'Nadakpan sa buy-bust ang ilang target nga mao usab ang nagpadagan sa balay suyopanan nga giila nga sila si Nolan Bigcas alyas Joshua, 42, nagmintinar sa suyopanan; Enan Biglang, 43, masahista; ug Junrex Siaboc Dalia, 38, drayber sa habalhabal.',
   'start': 323},
  'answer': {'end': 191, 'start': 182, 'text': 'masahista'},
  'pred': {'article_id': 200,
   'title': 'Drug den gironda, 3 nangasikop',
   'body': 'Usa ka drug den nga nahimutang sa Sitio Tabay Lawum, Barangay Tisa, Siyudad sa Sugbo ang gibungkag sa mga sakop sa Philippine Drug Enforcement Agency (PDEA) 7 uban sa Regional Intelligence Unit sa Police Regional Office (PRO) 7, alas 6:25 sa hapon, Sabado, Agusto 24, 2024, nga niresulta sa pagkasikop sa tulo ka mga tawo. Nadakpan sa buy-bust ang ilang target nga mao usab ang nagpadagan sa balay suyopanan nga giila nga sila si Nolan Bigcas alyas Joshua, 42, nagmintinar sa suyopa

In [108]:
qa.evaluate()


{'exact_match': 0.0, 'f1': 40.0, 'sentence_match': 0.0}


({'exact_match': 0.0, 'f1': 40.0, 'sentence_match': 0.0},
 {'model_path': '/Users/jhoannaricalagumbay/Library/CloudStorage/GoogleDrive-jtlagumbay@up.edu.ph/My Drive/UP Files/IV - 2nd sem/CMSC 198.1/cebqa_roberta/xlmr/2025-03-15_09-36/model',
  'tokenizer_path': '/Users/jhoannaricalagumbay/Library/CloudStorage/GoogleDrive-jtlagumbay@up.edu.ph/My Drive/UP Files/IV - 2nd sem/CMSC 198.1/cebqa_roberta/xlmr/2025-03-15_09-36/tokenizer',
  'k': 1,
  'sample': 1,
  'isRandom': True,
  'overlap': 0.0,
  'num_chunks': 3},
 {'run_time': 0.318248987197876})

In [54]:
results = qa.results
preds = [
        {
            'id': result['id'],  # Convert ID to string
            'prediction_text': qa.normalize_text(result['pred']['answer'])
        }
        for result in qa.results
    ]

refs = [
    {
        'id': item['id'],  # Convert ID to string
        'answers': {
            'text': [qa.normalize_text(item['answer']['text'])],
            'answer_start': [item['answer']['start']]
        }
    }
    for item in qa.results
]

em_count = 0
sm_count = 0
for index, item in enumerate(preds):
    if item['prediction_text'] == refs[index]['answers']['text'][0]:
        em_count += 1
    if item['prediction_text'] in refs[index]['answers']['text'][0]:
        sm_count += 1

print(em_count, sm_count)

0 1


# Retriever Performance

In [ ]:
dataset = load_dataset(CEBQA_DATASET)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
test_dataset = dataset["test"]
test_dataset

Dataset({
    features: ['id', 'article_id', 'article_title', 'article_body', 'question', 'context', 'answer', 'context_start', 'context_end', 'answer_start', 'answer_end'],
    num_rows: 5526
})

In [19]:
import torch
print(torch.backends.mps.is_available())  # Should return True
print(torch.backends.mps.is_built())


True
True


In [ ]:
retriever = BM25Retriever()
correct_count = 0
wrong_count = 0
wrong_rows = []
for row in test_dataset:
  print(f"processing {row['id']}")
  query = row["question"]
  top_docs = retriever.retrieve(query, K)

  if row["article_id"] in [doc["id"] for doc in top_docs]:
    correct_count+=1
  else:
    wrong_count+=1
    wrong_rows.append(row)

print(f"correct: {correct_count}")
print(f"wrong: {wrong_count}")

processing 01437-013
processing 01130-018
processing 00437-006
processing 00247-009


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



processing 00101-004
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-ced425423ec3>", line 8, in <cell line: 0>
    top_docs = retriever.retrieve(query, K)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-11-253a20d7bee3>", line 8, in retrieve
    response = self.es.search(index=self.index_name, body={
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/elasticsearch/_sync/client/utils.py", line 455, in wrapped
    return api(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/elasticsearch/_sync/client/__init__.py", line 5061, in search
    return self.perform_request(  # type: ignore[return-value]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-

TypeError: object of type 'NoneType' has no len()

In [ ]:
print([row["id"] for row in wrong_rows])

## Batch Retrieving

In [ ]:
retriever = BM25Retriever()
batch_size = 100
top_k = 1
top_docs = []
correct_count = 0
wrong_count = 0
wrong_rows = []

for i in range(0, len(test_dataset), batch_size):
    batch = test_dataset.select(range(i, min(i + batch_size, len(test_dataset))))

    # Retrieve documents in batch
    batch_top_docs = retriever.retrieve_batch([row["question"] for row in batch], top_k)

    for row, top_docs in zip(batch, batch_top_docs):
        if row["article_id"] in [doc["id"] for doc in top_docs]:
            correct_count += 1
        else:
            wrong_count += 1
            wrong_rows.append(row)

print(f"correct: {correct_count}")
print(f"wrong: {wrong_count}")
print([row["id"] for row in wrong_rows])

## Questions that are always wrong

In [ ]:
wrong_1 = ['00437-006', '00247-009', '01195-010', '00139-012', '00206-016', '00561-015', '00158-008', '01024-009', '00123-018', '00647-013', '00982-008', '01305-006', '00565-016', '01545-011', '01164-010', '00509-015', '01022-001', '01690-017', '00192-014', '01208-009', '01508-002', '00574-016', '00937-010', '00719-017', '01393-013', '00475-002', '00107-008', '00838-007', '00838-005', '00843-010', '01632-008', '00474-006', '00711-009', '01245-015', '01003-014', '00078-013', '01390-011', '00915-004', '00133-009', '00933-007', '00699-011', '00919-001', '00681-009', '01266-016', '01180-004', '00990-004', '00967-005', '01067-002', '00254-016', '01365-009', '00165-004', '00830-015', '00473-006', '00734-005', '00846-006', '00480-009', '00380-011', '01599-001', '00248-010', '01513-006', '00344-010', '00865-008', '01335-014', '01630-016', '00430-001', '00556-008', '01225-012', '00688-017', '00513-002', '00526-017', '01556-003', '00322-012', '01414-001', '00636-010', '00059-001', '00071-003', '00913-001', '01260-013', '01144-003', '01692-014', '00688-006', '01444-014', '00948-018', '01632-006', '01225-005', '00590-017', '00241-012', '01390-005', '00182-015', '00526-005', '00152-015', '01540-014', '00488-008', '00796-014', '00427-018', '00402-007', '00655-006', '01622-016', '01325-012', '00887-015', '00790-018', '00415-006', '01639-001', '00072-002', '00530-014', '00263-013', '01608-010', '00799-002', '00226-005', '00491-010', '00569-011', '01029-012', '00545-002', '01301-007', '00625-003', '01168-010', '00110-005', '00772-012', '01268-017', '00821-017', '00131-012', '00128-013', '00313-017', '01442-011', '01328-007', '01130-013', '01161-013', '00784-017', '01422-010', '00371-008', '00889-002', '01353-014', '01546-017', '01394-004', '00949-016', '01325-006', '00509-002', '01445-017', '00938-008', '00905-002', '01321-014', '00855-008', '00113-015', '00484-007', '01615-001', '01371-004', '00746-011', '01254-007', '00395-006', '00317-007', '00069-014', '00600-016', '01061-003', '00448-010', '00079-012', '01208-006', '00126-010', '01059-015', '00720-005', '00055-016', '00567-017', '01623-005', '01230-009', '01568-002', '00726-013', '00703-003', '00752-013', '00535-018', '00452-006', '00651-017', '00353-013', '01291-012', '00375-006', '00255-014', '01489-004', '01442-003', '00484-008', '01223-007', '00439-016', '00572-003', '01658-007', '01294-007', '00928-017', '01330-013', '00869-005', '01564-016', '00614-003', '00321-016', '00550-002', '01094-006', '00726-010', '01560-005', '01301-016', '01244-017', '00667-008', '00248-009', '01558-004', '00992-014', '01476-002', '00396-013', '00730-016', '00052-014', '00930-018', '01157-013', '01222-001', '01697-013', '01543-009', '00525-004', '01268-016', '01599-003', '00515-006', '00414-013', '00268-014', '01109-014', '01135-007', '00192-013', '00212-011', '01546-012', '00313-001', '00670-004', '00579-006', '01219-007', '00171-018', '01456-005', '00474-009', '00516-004', '00444-009', '01249-006', '00480-001', '01435-004', '00968-015', '00453-004', '00062-006', '00886-008', '00981-006', '00701-012', '00031-012', '01625-010', '01542-012', '01241-015', '01402-003', '00499-004', '00169-004', '00519-011', '00525-007', '01144-007', '00589-008', '01256-010', '00113-012', '01378-016', '01030-006', '00557-014', '00257-005', '01560-003', '00392-015', '00330-010', '00133-002', '01100-010', '00623-011', '00478-013', '01007-004', '00907-010', '01129-011', '00322-006', '00708-017', '01311-012', '00850-016', '00143-003', '00525-012', '00193-009', '00226-004', '00037-012', '00280-016', '01291-004', '01301-001', '00505-017', '01360-014', '01610-010', '00453-005', '00164-001', '00257-017', '00560-012', '00517-004', '00610-003', '00044-012', '00748-008', '00513-003', '00706-003', '00422-013', '00617-011', '00152-002', '00725-011', '00943-011', '00212-006', '00231-011', '00471-014', '00582-013', '01547-009', '01451-009', '01071-004', '01382-002', '01134-011', '00179-013', '01521-005', '00934-018', '01552-013', '01627-008', '01377-017', '00213-003', '01525-015', '01227-008', '00283-011', '00813-013', '01545-001', '00525-014', '01276-012', '00460-015', '00471-017', '00230-013', '00931-007', '01073-014', '00793-003', '01036-016', '01443-016', '00480-010', '00395-003', '00233-014', '00901-015', '01420-016', '01263-001', '01324-006', '01440-013', '00335-004', '01632-004', '00589-007', '00577-014', '01408-004', '01592-003', '01107-010', '01210-006', '01555-002', '01139-017', '00505-014', '01214-011', '01567-005', '01349-012', '00232-011', '00709-014', '00183-014', '00230-006', '00744-010', '00928-018', '00732-003', '01230-008', '01444-003', '01152-009', '01567-018', '00360-016', '00241-009', '00197-011', '01453-015', '00919-016', '00703-008', '01418-016', '00868-009', '01084-001', '00186-001', '00048-017', '01425-012', '00295-014', '00197-006', '00563-006', '00343-001', '00788-011', '01439-018', '00059-003', '00694-014', '01143-017', '01109-006', '01036-002', '01135-012', '00687-005', '00048-006', '00946-016', '01338-007', '00849-011', '00747-012', '01380-002', '00085-014', '00974-013', '00587-013', '00745-005', '01406-002', '00599-003', '00583-013', '01083-016', '01071-011', '01161-016', '00085-007', '00050-013', '01180-006', '00806-015', '01596-015', '00752-015', '01096-017', '01189-015', '01370-014', '00671-011', '01103-011', '00255-008', '01559-014', '00085-005', '00316-017', '00821-016', '01227-012', '00363-011', '00718-008', '01534-015', '00932-003', '00396-007', '01591-015', '01669-001', '01362-010', '00631-003', '00732-009', '00159-007', '01291-016', '00904-001', '00449-013', '00763-016', '01024-005', '00510-004', '00999-008', '00928-005', '01315-009', '00561-005', '01190-007', '00953-011', '00972-016', '01319-005', '00714-013', '00721-011', '00268-013', '01250-013', '00123-004', '00143-012', '01358-017', '00368-002', '01608-001', '01532-007', '00850-009', '01218-005', '01520-018', '00831-006', '01544-011', '00421-011', '01547-017', '00477-006', '01583-003', '01041-013', '00463-002', '01420-015', '00972-002', '01615-010', '00200-012', '00572-015', '00761-014', '00289-004', '00241-007', '01263-007', '00983-003', '00047-003', '00156-017', '00600-013', '01396-010', '01355-003', '00042-001', '01113-003', '00765-010', '00072-009', '00720-015', '01284-014', '01245-013', '01557-011', '00083-002', '01419-011', '00250-010', '00347-006', '00072-014', '01059-004', '00037-003', '01669-015', '00132-011', '01559-017', '00988-007', '00353-007', '00555-007', '00729-014', '00934-005', '01549-007', '00394-002', '01145-017', '00256-010', '01508-003', '01287-009', '01276-009', '00572-002', '00164-014', '01059-012', '01379-017', '01325-002', '01139-016', '00956-010', '00595-013', '01372-005', '00211-002', '01675-002', '00289-005', '00134-016', '00209-001', '00851-001', '00701-013', '00445-002', '00142-011', '00799-018', '00846-015', '00789-015', '01596-004', '00899-003', '01623-015', '01034-013', '01380-010', '00296-009', '01039-014', '00933-005', '01264-013', '00869-008', '01288-004', '00525-008', '01388-011', '00793-017', '01312-009', '00360-017', '00931-008', '00943-015', '00559-016', '01326-012', '01264-017', '00643-005', '01274-008', '00092-011', '01329-010', '00214-018', '00776-009', '00937-017', '01564-010', '01094-010', '01632-011', '00772-011', '00677-002', '00453-015', '00943-006', '01662-002', '00983-007', '01463-009', '00642-004', '01072-016', '00486-002', '00852-015', '00322-009', '00513-013', '00769-016', '01224-017', '00408-012', '01144-016', '00804-002', '00671-012', '01114-009', '00871-016', '01145-003', '00078-008', '00816-003', '00469-009', '01563-009', '01100-009', '01420-012', '00896-002', '00946-014', '00820-016', '00085-002', '01029-016', '00059-010', '00448-004', '01153-005', '00871-010', '00492-001', '01578-007', '01335-011', '00645-009', '01004-003', '01627-004', '00108-013', '00087-001', '00730-007', '00621-015', '01275-009', '00051-004', '00180-015', '01391-010', '00606-013', '01196-012', '01401-001', '01378-003', '00445-017', '00294-002', '01631-009', '00760-010', '00352-004', '01378-010', '00619-008', '00663-003', '00034-012', '00500-012', '01216-014', '00938-004', '00030-006', '01681-013', '00156-004', '00037-001', '01563-016', '00819-003', '00397-002', '01613-009', '00393-013', '01683-013', '01042-009', '01564-004', '00442-008', '00765-007', '00209-007', '01111-012', '00460-002', '01517-008', '00038-008', '00884-001', '00726-015', '01549-006', '01353-001', '00940-009', '00302-015', '01275-010', '00620-003', '01111-010', '00754-004', '00938-010', '01406-010', '00214-010', '00096-016', '00772-005', '01167-012', '01142-007', '00394-007', '01368-011', '00647-004', '01662-014', '01100-013', '01308-016', '01688-015', '01325-016', '01329-007', '00192-004', '00580-015', '00833-006', '00247-011', '00938-011', '01005-017', '00452-010', '00721-012', '01559-005', '00569-009', '01082-016', '00606-009', '00826-004', '00915-017', '01378-008', '00845-004', '00556-001', '00078-019', '01227-017', '01096-016', '00817-011', '00831-011', '00729-004', '00073-017', '01673-017', '00427-011', '00452-007', '01696-009', '00555-012', '00994-010', '00427-003', '00543-017', '00808-009', '00966-008', '00954-005', '00351-002', '00059-016', '00694-002', '00991-004', '00213-008', '01305-010', '01623-012', '00406-015', '00779-011', '01344-015', '01277-012', '01199-004', '00699-016', '00209-005', '01046-001', '01511-016', '00498-011', '00537-017', '00808-015', '01174-013', '01631-001', '00351-007', '01618-012', '00889-003', '00523-007', '00192-008', '01404-015', '00743-014', '00363-016', '00214-014', '01558-005', '00242-002', '00028-005', '01406-004', '00327-013', '01117-001', '01047-018', '00480-017', '00589-011', '01605-008', '01034-014', '01082-014', '00940-013', '00450-009', '00181-010', '00193-006', '01417-008', '01165-014', '00680-005', '00076-008', '01464-007', '01310-003', '00759-016', '01265-016', '00291-013', '00038-014', '00194-015', '00982-017', '00971-013', '01237-010', '00534-016', '00444-003', '00638-004', '00651-001', '00931-010', '01443-017', '01380-013', '00627-003', '00483-012', '00699-012', '01581-004', '00077-002', '00691-015', '01256-012', '01140-007', '01113-013', '01421-003', '01684-017', '00407-017', '00365-014', '00515-013', '00480-018', '00797-014', '01095-014', '01147-002', '00036-009', '01489-010', '00439-005', '01020-015', '01146-013', '00463-015', '00458-002', '00596-009', '01018-009', '00701-006', '00843-011', '00303-011', '01674-014', '01388-006', '00647-012', '01371-015', '00743-004', '00472-002', '01615-008', '01622-008', '01264-007', '01596-006', '00217-018', '00766-015', '01511-004', '01312-018', '00384-004', '00311-008', '00343-015', '01554-013', '00227-014', '00423-017', '00352-005', '01574-002', '00356-013', '00383-012', '00034-009', '01296-011', '00825-003', '01021-014', '01495-014', '01674-016', '00203-009', '00699-002', '00402-002', '01379-008', '01227-003', '01109-010', '00362-005', '00093-010', '00913-011', '01111-016', '00649-004', '00025-013', '00970-003', '00582-011', '00128-004', '00159-015', '00961-016', '01522-008', '01418-003', '00605-012', '00499-008', '00793-002', '01419-002', '00686-012', '01237-004', '01546-015', '01532-014', '01181-004', '00897-001', '00474-001', '00391-007', '00889-004', '00379-014', '00047-015', '00901-011', '01292-014', '00851-002', '01556-012', '01676-002', '00278-007', '00222-013', '00843-003', '00643-001', '01631-011', '01644-006', '00296-015', '00750-016', '00089-012', '00189-015', '01369-004', '01335-017', '00993-010', '00448-012', '00732-005', '01552-006', '00486-010', '00029-006', '01684-002', '01002-008', '00732-004', '01674-011', '01442-004', '01248-012', '00711-011', '00990-006', '00510-007', '00125-009', '00198-019', '01487-016', '00918-005', '00335-002', '01099-015', '01669-007', '01480-007', '00194-006', '01024-012', '00645-007', '01388-012', '01152-008', '01548-017', '01157-005', '01315-001', '01378-015', '01571-012', '01670-015', '00698-006', '00611-001', '01496-005', '00065-007', '01384-005', '01676-010', '00451-004', '01208-004', '01257-010', '00492-004', '01421-012', '00837-010', '00267-001', '00775-002', '00541-007', '00399-013', '01384-013', '01223-008', '01447-012', '01539-012', '00209-003', '01459-012', '00785-018', '00562-003', '01101-005', '01138-014', '00603-006', '00705-006', '00085-015', '01477-010', '00628-004', '01650-002', '00425-012', '00940-003', '01529-010', '01671-008', '01627-009', '01170-012', '00474-002', '00580-014', '00396-016', '00589-013', '00434-006', '00525-015', '00513-017', '01169-009', '01654-016', '00374-009', '00390-003', '00256-002', '00617-015', '01134-001', '00742-016', '00180-008', '01210-011', '01189-009', '00825-002', '01272-016', '01533-005', '00313-005', '01484-016', '00842-014', '01458-001', '01343-002', '00300-003', '01680-015', '01113-006', '00180-012', '01483-004', '01112-004', '01386-013', '00504-011', '00904-002', '00834-003', '00665-005', '01609-016', '00192-007', '00918-006', '00528-010', '01556-001', '01134-006', '00879-016', '00375-007', '00695-005', '00918-002', '00541-018', '00943-012', '00990-010', '00047-010', '00865-007', '01170-007', '01613-004', '01149-002', '00501-012', '00817-001', '00994-011', '00338-013', '01236-006', '01013-018', '00441-013', '00091-002', '00801-010', '00354-001', '01400-006', '01068-015', '01252-013', '01039-009', '01470-014', '00992-016', '01439-017', '00221-014', '00181-015', '01029-011', '01258-004', '01394-007', '01117-004', '01465-015', '01598-006', '00600-017', '00400-003', '01170-006', '01675-017', '01029-006', '01211-011', '00668-011', '01465-018', '01674-002', '01118-002', '00639-011', '01143-010', '01059-007', '00847-016', '01445-002', '01491-007', '00930-003', '00179-001', '00647-011', '00425-018', '00207-008', '00567-010', '00521-007', '00953-012', '00360-002', '01300-004', '00915-014', '00749-002', '00457-016', '00126-014', '00938-017', '01229-014', '00712-006', '00894-002', '01643-011', '01071-007', '00938-005', '01319-015', '01238-014', '00232-003', '01264-012', '01396-015', '00078-009', '01032-001', '00695-004', '00510-001', '00869-014', '00976-004', '00066-011', '00087-003', '01165-015', '00242-015', '00649-003', '00490-012', '00302-001', '00583-003', '00180-007', '00073-013', '01089-009', '00447-009', '01084-004', '00038-006', '01046-015', '01310-011', '01523-008', '01505-002', '01001-016', '00594-008', '01651-013', '00582-009', '00627-008', '01118-013', '00131-010', '00696-001', '01275-001', '00183-004', '00990-017', '00900-018', '00673-008', '01553-014', '01696-016', '01137-008', '00357-003', '00048-008', '00835-009', '00211-017', '00469-001', '00645-014', '01458-002', '00777-012', '00822-004', '01401-002', '00284-011', '00321-019', '01465-005', '01223-019', '00873-007', '00604-013', '00745-001', '00452-016', '00193-001', '00029-017', '00178-001', '00563-014', '00798-016', '01039-005', '01253-002', '00700-011', '00502-007', '01679-004', '00751-013', '01090-011', '00650-010', '00655-004', '01250-008', '01550-016', '01699-003', '00247-014', '01013-011', '01200-016', '00400-013', '00502-013', '00131-001', '00064-011', '01553-005', '01522-009', '01529-001', '00615-006', '01272-007', '01022-003', '00275-008', '01325-003', '01060-011', '00362-016', '00499-007', '00575-002', '00116-013', '00836-002', '00394-003', '01149-017', '01646-019', '00464-009', '00992-002', '01114-005', '01379-007', '01420-006', '00927-017', '01292-006', '00229-008', '00090-008', '00803-015', '01506-015', '00062-004', '00543-016', '00522-004', '01140-001', '01244-002', '01580-019', '01310-017', '00833-009', '00463-008', '00552-007', '01370-013', '00093-015', '00826-014', '01370-009', '00941-003', '01104-016', '01457-008', '01187-010', '00437-003', '00779-010', '00656-017', '00913-003', '00510-006', '00091-008', '00469-010', '00357-017', '00776-002', '01021-006', '00510-016', '00104-002', '00931-003', '01530-006', '01170-016', '00300-011', '00116-007', '00403-002', '00950-001', '00226-010', '00593-016', '00653-015', '01572-002', '00188-007', '00075-004', '00176-003', '01479-012', '01142-008', '00760-017', '01445-012', '00923-003', '01353-002', '00114-007', '00371-012', '00254-013', '00636-013', '01361-016', '00201-015', '01696-003', '01388-005', '00500-011', '01264-010', '01499-003', '00825-017', '01501-016', '00767-002', '01141-009', '00683-006', '00674-008', '00839-015', '00420-004', '00748-001', '01169-008', '01215-008', '01100-014', '00464-003', '00619-013', '01242-011', '01339-015', '00267-003', '01603-002', '00769-010', '00075-008', '00567-002', '00218-014', '00744-016', '01281-016', '01650-007', '00754-013', '01167-004', '00120-009', '00073-001', '01244-015', '00249-007', '01026-009', '01278-010', '00233-001', '00386-001', '00221-016', '00205-002', '01187-005', '01600-013', '00349-001', '00294-003', '00861-011', '00366-010', '01511-002', '00450-006', '01024-003', '00804-017', '01149-005', '01603-009', '00576-016', '00925-007', '01631-004', '00449-002', '00263-016', '00643-003', '00756-008', '00381-015', '00610-012', '00564-008', '01471-004', '00730-011', '01227-002', '01401-012', '00492-009', '00264-002', '00064-016', '00142-001', '00121-011', '01484-017', '00633-005', '01552-008', '01265-001', '01514-001', '00429-001', '00328-003', '00181-003', '01186-001', '00828-010', '00699-017', '01039-008', '00400-017', '00057-009', '00843-002', '00865-014', '00062-010', '00686-008', '01000-010', '00980-012', '00628-012', '00716-010', '00247-016', '01199-017', '00509-017', '01052-012', '01629-014', '00836-011', '00283-008', '01152-002', '01631-015', '00075-017', '01469-009', '00226-002', '00392-008', '00743-003', '01155-013', '01406-015', '01521-006', '00665-002', '01260-006', '00486-016', '01370-010', '00053-013', '01364-016', '00639-002', '00909-002', '01650-017', '00185-016', '01375-013', '01547-013', '01553-009', '00962-017', '01061-011', '01373-005', '00341-003', '00623-014', '00963-002', '01557-008', '01336-004', '00399-001', '01206-017', '01029-009', '00557-002', '00209-010', '01534-002', '01546-011', '00390-014', '01661-003', '00729-011', '00628-011', '00736-003', '01431-003', '01160-014', '00091-013', '00189-014', '00472-008', '00869-016', '01423-002', '00606-008', '00362-006', '00248-007', '01556-015', '00753-015', '01237-013', '00976-012', '01204-015', '01209-003', '00251-010', '00684-015', '00347-007', '01631-017', '00524-016', '00813-010', '01132-013', '01572-001', '00704-004', '01095-010', '01195-012', '00857-007', '00640-003', '00474-016', '00753-018', '01134-003', '01684-012', '00981-015', '00525-003', '01237-002', '01489-002', '00538-004', '01668-013', '00449-003', '00831-016', '01676-005', '00482-004', '01321-010', '00993-008', '01087-001', '00128-011', '00392-004', '00859-004', '01275-002', '01544-001', '00440-003', '01315-012', '01086-001', '01514-002', '00059-008', '00730-006', '01281-005', '00720-013', '00906-014', '01429-004', '01507-004', '01209-009', '01500-019', '01594-014', '01622-009', '00953-002', '00979-007', '00972-017', '01225-016', '00086-001', '00620-004', '00627-009', '01342-013', '00752-005', '01600-006', '00395-010', '01157-006', '01136-008', '00597-009', '00427-008', '00395-012', '00263-008', '00534-003', '01256-003', '01111-006', '00292-007', '01209-013', '00289-012', '00927-013', '01328-010', '01380-001', '00242-010', '00681-003', '01111-002', '00475-010', '00913-004', '00490-014', '00146-013', '00485-008', '00763-008', '00303-016', '01325-017', '01069-010', '00403-008', '00730-001', '00167-002', '00705-016', '01186-015', '00209-016', '00428-013', '00612-007', '00542-015', '00365-013', '01265-013', '00665-008', '00710-017', '00812-005', '01340-012', '01552-009', '00399-016', '00910-003', '01640-012', '00394-005', '00599-002', '00232-001', '00629-014', '01211-017', '00412-009', '01644-005', '00188-001', '01243-014', '00292-014', '00085-008', '01051-002', '01337-003', '00481-014', '01276-013', '00064-014', '01487-008', '01475-018', '00582-002', '00487-014', '01112-006', '00799-004', '00028-016', '00176-008', '00041-006', '00913-006', '01291-014', '01018-004', '01082-010', '01128-004', '01359-013', '00604-012', '00424-007', '01547-007', '00776-006', '00538-017', '00319-009', '00114-014', '01001-018', '00615-019', '01125-006', '00750-005', '01109-005', '00486-017', '00336-003', '01558-017', '00889-012', '01594-012', '00952-011', '00786-016', '00089-015', '01152-010', '01441-006', '00424-009', '00967-003', '00910-011', '00520-014', '01533-009', '01537-001', '00466-003', '00442-011', '01207-015', '00158-009', '01668-017', '00753-010', '01522-012', '00402-005', '01272-018', '00155-005', '01111-009', '01260-012', '01436-016', '00241-003', '01143-014', '00603-007', '00821-015', '00981-007', '01099-004', '01654-001', '00289-013', '01534-007', '00036-015', '00482-005', '01069-003', '00781-015', '01676-011', '01399-003', '00925-013', '01529-003', '00864-010', '01121-008', '00072-011', '00926-006', '00034-017', '01253-003', '00941-006', '01054-012', '00333-011', '00993-014', '00096-004', '00767-003', '00276-010', '00908-006', '01658-008', '01445-001', '01516-013', '00089-011', '01553-003', '00078-016', '00502-017', '00209-013', '00788-015', '00926-008', '00026-002', '00651-010', '01020-002', '01556-014', '00677-004', '01412-011', '00335-015', '01580-010', '01037-009', '00120-014', '00948-007', '01482-016', '01291-007', '00171-001', '00595-015', '00361-003', '00861-004', '00963-015', '00028-014', '01501-014', '01407-004', '00735-016', '01225-006', '00083-012', '00360-009', '01673-015', '00745-006', '00736-004', '00792-012', '00550-018', '01483-014', '00971-012', '01101-002', '00091-010', '01028-014', '01662-016', '00611-009', '00531-006', '01114-002', '00531-016', '00490-009', '01650-006', '01622-012', '00780-013', '00120-003', '00423-011', '00684-006', '00969-010', '01303-003', '00420-014', '00302-012', '00755-016', '01522-013', '00403-006', '00549-001', '00825-010', '00642-013', '01646-018', '00473-008', '00194-009', '01069-016', '01280-005', '01534-008', '01100-016', '01630-002', '01343-004', '00188-008', '00313-015', '00256-017', '01211-013', '00368-006', '00054-007', '00686-002', '01292-011', '00796-004', '01206-013', '00034-005', '01071-014', '01345-006', '00749-009', '00681-011', '00869-007', '00669-006', '00617-004', '00760-008', '00040-010', '00424-008', '00683-009', '00406-012', '01212-014', '00569-013', '01321-002', '00068-002', '00415-005', '00064-006', '00913-005', '01445-008', '01264-006', '00130-004', '01471-009', '00477-007', '00915-002', '01615-003', '01268-006', '01272-012', '00125-002', '01560-007', '01553-016', '01359-005', '00987-011', '00093-014', '01164-013', '00995-007', '01586-016', '01631-007', '00958-015', '00373-017', '00906-010', '01632-013', '00854-004', '00454-015', '00927-011', '01146-009', '00143-001', '00883-014', '01640-001', '01263-017', '00982-006', '01391-008', '01301-010', '00595-001', '00837-001', '00196-015', '00986-014', '01456-016', '00592-017', '00648-016', '00412-013', '01516-011', '00883-003', '01099-011', '01574-001', '00260-014', '01548-012', '00774-003', '00346-012', '00933-008', '01668-003', '00776-015', '01291-001', '01303-006', '00962-010', '00226-011', '01442-014', '00241-008', '00972-001', '01221-005', '01594-015', '00868-013', '01199-005', '01026-006', '01366-015', '01590-008', '00328-010', '00683-017', '00647-001', '00548-012', '00517-007', '01598-014', '00720-010', '00311-005', '01192-015', '00821-019', '01555-016', '01284-001', '00950-005', '01063-014', '01597-014', '00074-010', '00242-001', '00054-018', '00573-002', '00302-009', '01282-017', '01566-007', '00146-016', '00095-018', '00885-012', '00870-009', '00490-016', '00213-007', '00752-017', '01517-012', '01493-014', '01268-014', '01076-006', '01361-008', '00108-004', '01296-004', '01082-018', '00491-014', '01220-003', '00571-016', '00331-016', '00115-003', '00292-005', '01018-016', '00320-004', '00118-006', '00562-002', '01362-009', '00919-007', '01410-013', '01341-011', '00572-004', '00205-004', '00617-014', '01508-005', '00440-015', '01444-004', '01168-002', '00032-009', '01110-008', '00857-005', '00775-003', '00642-005', '01181-003', '00263-015', '01375-003', '00573-017', '00950-017', '00414-017', '01043-013', '01113-002', '01026-005', '01120-001', '01687-008', '00469-014', '01033-001', '01224-009', '00618-010', '00929-009', '00409-001', '01220-007', '00938-003', '01313-012', '01369-014', '01014-003', '01123-015', '01585-017', '01112-016', '01362-004', '00639-012', '00525-010', '01284-011', '00569-007', '00704-008', '00255-005', '01263-015', '01088-001', '00722-001', '00928-014', '00932-017', '01200-010', '00681-015', '00938-009', '00077-015', '00091-009', '00812-017', '00835-005', '00313-012', '01186-012', '01620-017', '00078-003', '00848-001', '00716-005', '00455-013', '00128-005', '00200-002', '01677-002', '00415-011', '00225-002', '00277-016', '01642-009', '00062-016', '01681-008', '01689-003', '01125-016', '01258-017', '01557-016', '00761-010', '01382-001', '01169-016', '00308-016', '01078-006', '00086-013', '01075-015', '00592-003', '00627-015', '01558-014', '00130-002', '00906-006', '00950-004', '01255-003', '01217-013', '00480-015', '00837-018', '00386-015', '01263-003', '01048-002', '00098-003', '01105-001', '00412-017', '00977-012', '01120-006', '01188-006', '01199-011', '01402-011', '00481-010', '00956-015', '00815-004', '00095-012', '01129-015', '00044-005', '00923-016', '01430-017', '00031-006', '00081-003', '00930-007', '00156-010', '01276-001', '01048-003', '00799-017']
wrong_3 = ['00247-009', '01024-009', '00123-018', '00647-013', '00982-008', '01305-006', '01545-011', '01022-001', '01690-017', '00192-014', '01208-009', '01508-002', '00574-016', '00719-017', '01393-013', '00475-002', '00107-008', '00838-007', '00838-005', '00843-010', '01632-008', '00711-009', '01003-014', '00078-013', '00133-009', '00933-007', '00919-001', '00681-009', '01266-016', '01180-004', '01067-002', '00254-016', '00165-004', '00473-006', '00480-009', '00380-011', '01599-001', '01513-006', '00865-008', '01335-014', '00430-001', '00556-008', '00688-017', '00513-002', '00526-017', '00322-012', '01414-001', '00636-010', '00071-003', '00913-001', '01260-013', '01144-003', '01692-014', '01444-014', '01225-005', '00590-017', '00241-012', '01390-005', '00182-015', '00152-015', '00796-014', '00402-007', '00655-006', '01622-016', '01325-012', '00887-015', '00790-018', '00415-006', '00072-002', '00530-014', '00263-013', '01608-010', '00799-002', '00226-005', '00491-010', '00569-011', '01029-012', '01168-010', '00110-005', '00772-012', '00821-017', '00131-012', '00128-013', '01442-011', '01130-013', '01161-013', '00784-017', '01422-010', '00889-002', '01353-014', '01546-017', '00949-016', '01325-006', '01445-017', '00855-008', '01615-001', '01371-004', '00746-011', '00069-014', '00600-016', '01061-003', '00448-010', '00079-012', '01208-006', '00126-010', '00055-016', '00567-017', '01623-005', '01568-002', '00726-013', '00703-003', '00752-013', '00535-018', '00452-006', '00651-017', '01291-012', '00255-014', '01489-004', '01442-003', '00484-008', '01223-007', '00572-003', '01294-007', '00928-017', '01330-013', '00869-005', '01564-016', '00321-016', '00550-002', '01094-006', '00726-010', '01560-005', '01301-016', '01244-017', '00248-009', '01558-004', '00992-014', '00396-013', '00730-016', '00052-014', '00930-018', '01157-013', '01697-013', '00525-004', '01599-003', '00515-006', '00414-013', '00268-014', '01109-014', '00192-013', '01546-012', '00579-006', '01456-005', '00474-009', '00516-004', '00444-009', '01249-006', '00480-001', '01435-004', '00453-004', '00062-006', '00886-008', '00981-006', '00031-012', '01625-010', '01542-012', '01241-015', '01402-003', '00499-004', '00519-011', '01144-007', '00113-012', '01378-016', '01030-006', '00557-014', '01560-003', '00392-015', '00330-010', '00133-002', '00623-011', '01007-004', '01129-011', '00322-006', '01311-012', '00850-016', '00143-003', '00525-012', '00193-009', '00226-004', '00037-012', '00280-016', '01291-004', '01301-001', '00505-017', '01360-014', '00453-005', '00164-001', '00257-017', '00560-012', '00517-004', '00610-003', '00513-003', '00706-003', '00422-013', '00617-011', '00725-011', '00943-011', '00231-011', '00471-014', '00582-013', '01547-009', '01451-009', '01071-004', '01382-002', '01134-011', '00179-013', '01521-005', '01552-013', '00213-003', '01525-015', '01227-008', '00283-011', '00525-014', '01276-012', '00460-015', '00230-013', '00931-007', '01073-014', '00793-003', '01036-016', '01443-016', '00480-010', '00395-003', '00233-014', '00901-015', '01420-016', '01263-001', '01440-013', '00589-007', '00577-014', '01592-003', '01107-010', '01210-006', '00505-014', '01214-011', '01567-005', '01349-012', '00232-011', '00709-014', '00230-006', '00744-010', '00928-018', '00732-003', '01444-003', '01567-018', '00360-016', '00241-009', '00197-011', '00919-016', '01418-016', '00048-017', '00197-006', '00343-001', '00788-011', '01439-018', '01143-017', '01109-006', '01036-002', '00687-005', '00048-006', '00946-016', '01338-007', '00849-011', '01380-002', '00085-014', '01406-002', '00599-003', '00583-013', '01083-016', '01071-011', '01161-016', '00085-007', '00050-013', '01180-006', '00752-015', '01096-017', '01189-015', '00671-011', '00255-008', '00316-017', '00821-016', '00363-011', '01534-015', '00396-007', '01591-015', '01669-001', '00631-003', '00732-009', '00449-013', '00763-016', '00510-004', '00999-008', '00928-005', '00561-005', '01319-005', '00721-011', '00268-013', '01250-013', '00143-012', '01358-017', '00368-002', '01608-001', '00850-009', '01520-018', '01544-011', '01547-017', '00477-006', '01583-003', '00463-002', '01420-015', '00972-002', '00200-012', '00572-015', '00761-014', '00241-007', '00983-003', '00047-003', '00156-017', '01396-010', '01355-003', '00042-001', '01113-003', '00765-010', '00072-009', '00720-015', '01284-014', '00083-002', '01419-011', '00250-010', '00347-006', '00072-014', '01059-004', '00037-003', '01669-015', '00132-011', '01559-017', '00353-007', '00555-007', '00934-005', '00394-002', '01145-017', '00256-010', '01508-003', '00572-002', '01059-012', '01379-017', '01325-002', '00956-010', '00595-013', '00211-002', '01675-002', '00289-005', '00209-001', '00701-013', '00445-002', '00142-011', '00799-018', '00846-015', '00789-015', '00899-003', '01623-015', '01034-013', '01380-010', '01039-014', '00933-005', '01264-013', '01288-004', '00525-008', '01388-011', '00793-017', '01312-009', '00931-008', '00943-015', '01326-012', '01264-017', '00643-005', '00776-009', '01632-011', '00772-011', '00453-015', '01662-002', '00983-007', '00642-004', '00486-002', '00852-015', '00513-013', '01224-017', '00671-012', '01114-009', '00871-016', '01145-003', '00078-008', '00816-003', '00469-009', '01420-012', '00896-002', '00946-014', '00085-002', '00448-004', '00871-010', '01578-007', '01335-011', '00645-009', '01004-003', '01627-004', '00087-001', '00730-007', '00621-015', '00051-004', '01391-010', '00606-013', '01196-012', '01401-001', '00445-017', '00294-002', '01631-009', '00760-010', '00352-004', '01378-010', '00500-012', '01216-014', '00030-006', '01681-013', '00156-004', '01563-016', '00397-002', '01613-009', '00393-013', '01683-013', '01564-004', '00209-007', '01111-012', '00460-002', '00038-008', '01549-006', '00940-009', '00302-015', '00620-003', '01111-010', '00754-004', '01406-010', '00096-016', '00772-005', '00394-007', '01368-011', '01662-014', '01100-013', '01688-015', '01325-016', '01329-007', '00192-004', '00580-015', '00247-011', '01005-017', '00452-010', '00721-012', '01559-005', '00569-009', '00845-004', '00556-001', '00078-019', '01227-017', '01096-016', '00817-011', '00729-004', '00073-017', '00427-011', '00452-007', '01696-009', '00555-012', '00427-003', '00543-017', '00954-005', '00351-002', '00694-002', '00991-004', '00213-008', '01305-010', '01623-012', '00406-015', '01344-015', '01199-004', '00699-016', '00209-005', '01046-001', '01511-016', '00498-011', '01174-013', '01631-001', '00351-007', '01618-012', '00889-003', '00523-007', '00192-008', '00743-014', '00363-016', '00214-014', '00242-002', '00028-005', '01406-004', '01117-001', '01047-018', '00589-011', '01605-008', '01034-014', '01082-014', '00940-013', '01417-008', '00680-005', '00076-008', '01265-016', '00291-013', '00038-014', '00194-015', '00534-016', '00444-003', '00651-001', '00931-010', '01443-017', '01380-013', '00627-003', '00483-012', '01581-004', '00077-002', '01140-007', '01113-013', '01421-003', '00365-014', '00480-018', '01095-014', '01147-002', '00036-009', '01489-010', '01018-009', '00303-011', '01674-014', '01388-006', '00647-012', '00743-004', '00472-002', '01615-008', '01622-008', '01264-007', '00217-018', '00766-015', '01312-018', '00311-008', '00343-015', '01554-013', '00227-014', '00423-017', '01574-002', '00383-012', '00034-009', '01296-011', '00825-003', '01495-014', '01674-016', '00699-002', '00402-002', '01379-008', '01109-010', '00362-005', '00093-010', '00913-011', '01111-016', '00649-004', '00128-004', '00961-016', '01522-008', '01418-003', '00605-012', '00499-008', '00793-002', '01419-002', '00686-012', '01237-004', '01546-015', '00897-001', '00474-001', '00391-007', '00889-004', '00379-014', '00047-015', '01292-014', '01556-012', '01676-002', '00222-013', '00843-003', '01631-011', '01644-006', '00296-015', '00750-016', '00089-012', '00189-015', '00993-010', '00448-012', '00732-005', '00486-010', '00029-006', '01684-002', '00732-004', '01674-011', '01442-004', '00711-011', '00990-006', '00510-007', '00198-019', '01487-016', '00918-005', '00335-002', '01099-015', '01669-007', '01024-012', '00645-007', '01152-008', '01157-005', '01378-015', '01571-012', '00698-006', '01496-005', '00065-007', '01384-005', '01676-010', '01208-004', '01257-010', '01421-012', '00267-001', '00775-002', '00541-007', '00399-013', '01384-013', '01539-012', '00209-003', '01459-012', '00785-018', '00562-003', '01101-005', '00603-006', '00705-006', '00085-015', '01477-010', '00628-004', '01650-002', '00425-012', '01529-010', '01170-012', '00580-014', '00396-016', '00589-013', '00513-017', '01169-009', '00390-003', '00256-002', '00617-015', '00180-008', '01210-011', '01272-016', '00313-005', '01484-016', '00842-014', '00300-003', '01680-015', '01113-006', '00180-012', '01483-004', '01112-004', '01386-013', '00904-002', '00834-003', '00665-005', '00192-007', '00918-006', '00528-010', '01134-006', '00879-016', '00695-005', '00918-002', '00541-018', '00943-012', '00990-010', '00047-010', '01613-004', '00501-012', '00817-001', '00338-013', '01236-006', '00441-013', '00801-010', '01068-015', '01039-009', '01470-014', '00221-014', '01258-004', '01394-007', '01117-004', '01465-015', '01598-006', '00600-017', '00400-003', '01170-006', '01675-017', '01029-006', '01211-011', '00668-011', '01465-018', '01674-002', '00639-011', '01143-010', '01059-007', '01445-002', '00930-003', '00179-001', '00425-018', '00360-002', '01300-004', '00749-002', '00457-016', '00126-014', '00894-002', '01643-011', '01071-007', '01319-015', '01238-014', '00232-003', '01264-012', '01032-001', '00869-014', '00087-003', '01165-015', '00649-003', '00490-012', '00302-001', '00583-003', '00180-007', '01084-004', '01046-015', '01310-011', '01523-008', '01001-016', '01651-013', '00582-009', '00627-008', '01118-013', '00990-017', '00900-018', '01696-016', '01137-008', '00048-008', '00835-009', '00211-017', '00469-001', '00645-014', '00777-012', '00822-004', '01401-002', '00321-019', '01465-005', '00604-013', '00745-001', '00452-016', '00029-017', '00178-001', '00798-016', '01039-005', '01253-002', '00700-011', '00502-007', '01679-004', '00751-013', '01090-011', '00650-010', '00655-004', '01250-008', '01200-016', '00400-013', '00502-013', '00131-001', '00064-011', '01553-005', '01522-009', '01529-001', '00615-006', '01272-007', '01022-003', '01325-003', '01060-011', '00362-016', '00499-007', '00116-013', '00836-002', '00394-003', '01646-019', '01114-005', '01379-007', '01420-006', '00927-017', '01292-006', '00229-008', '00090-008', '00803-015', '01506-015', '00062-004', '00543-016', '00522-004', '01140-001', '01244-002', '00463-008', '00552-007', '01370-013', '00093-015', '01104-016', '01457-008', '01187-010', '00437-003', '00779-010', '00656-017', '00913-003', '00510-006', '00091-008', '00776-002', '00510-016', '00104-002', '00931-003', '00300-011', '00116-007', '00403-002', '00950-001', '00226-010', '00593-016', '01572-002', '00075-004', '00176-003', '01479-012', '01142-008', '00760-017', '01445-012', '01353-002', '00371-012', '00254-013', '01361-016', '00201-015', '01388-005', '00500-011', '01264-010', '00825-017', '01501-016', '00767-002', '00674-008', '00839-015', '00748-001', '01169-008', '01215-008', '01100-014', '00267-003', '00769-010', '00075-008', '00218-014', '00744-016', '01650-007', '00754-013', '01167-004', '00120-009', '00249-007', '01026-009', '01278-010', '00233-001', '00386-001', '00221-016', '00205-002', '00861-011', '00450-006', '00804-017', '01149-005', '01603-009', '00576-016', '00925-007', '01631-004', '00449-002', '00263-016', '00643-003', '00610-012', '00564-008', '01471-004', '00730-011', '01401-012', '00264-002', '00064-016', '00142-001', '00121-011', '01484-017', '01552-008', '01265-001', '01514-001', '00328-003', '01186-001', '00828-010', '00699-017', '00865-014', '00062-010', '00686-008', '01000-010', '00628-012', '00247-016', '01199-017', '01052-012', '01629-014', '00836-011', '00283-008', '01152-002', '01631-015', '00075-017', '01469-009', '00226-002', '01406-015', '01260-006', '00486-016', '00053-013', '01364-016', '00639-002', '01650-017', '00185-016', '01547-013', '01373-005', '00623-014', '00963-002', '01557-008', '01336-004', '00399-001', '01206-017', '00557-002', '01534-002', '01546-011', '00390-014', '00729-011', '00628-011', '00736-003', '01431-003', '01160-014', '00091-013', '00189-014', '00472-008', '00869-016', '01423-002', '00606-008', '00362-006', '00248-007', '01556-015', '00753-015', '00976-012', '01204-015', '01209-003', '00251-010', '00684-015', '00347-007', '01631-017', '01132-013', '01572-001', '00704-004', '01095-010', '01195-012', '00640-003', '00474-016', '00753-018', '01134-003', '01684-012', '00981-015', '00525-003', '01237-002', '01489-002', '00538-004', '01668-013', '00449-003', '01676-005', '00482-004', '01321-010', '00993-008', '00128-011', '01275-002', '01544-001', '00440-003', '01315-012', '01514-002', '00059-008', '00906-014', '01429-004', '01507-004', '01209-009', '01500-019', '01594-014', '01622-009', '00972-017', '01225-016', '00086-001', '00620-004', '00627-009', '01342-013', '00752-005', '01600-006', '00395-010', '01157-006', '01136-008', '00597-009', '00427-008', '00395-012', '00263-008', '00534-003', '00292-007', '01209-013', '00927-013', '01380-001', '00681-003', '01111-002', '00475-010', '00913-004', '00490-014', '00146-013', '00485-008', '00763-008', '00303-016', '01325-017', '01069-010', '00403-008', '00730-001', '00167-002', '00705-016', '01186-015', '00209-016', '00542-015', '00365-013', '00665-008', '00812-005', '01340-012', '01552-009', '00399-016', '01640-012', '00232-001', '00629-014', '01211-017', '00412-009', '01644-005', '00188-001', '01243-014', '00292-014', '01051-002', '01276-013', '00064-014', '01475-018', '00582-002', '00487-014', '01112-006', '00799-004', '00028-016', '00176-008', '00913-006', '01291-014', '01018-004', '01082-010', '01128-004', '00424-007', '01547-007', '00776-006', '00538-017', '00114-014', '01001-018', '01109-005', '00486-017', '00336-003', '01558-017', '00889-012', '01594-012', '00786-016', '00089-015', '01152-010', '01533-009', '00442-011', '01207-015', '00158-009', '01668-017', '00753-010', '01522-012', '00402-005', '01272-018', '01111-009', '01260-012', '00241-003', '01143-014', '00603-007', '00821-015', '00981-007', '01099-004', '00289-013', '01534-007', '00036-015', '00482-005', '01069-003', '00781-015', '01676-011', '00925-013', '00072-011', '00926-006', '00034-017', '01253-003', '00941-006', '01054-012', '00333-011', '00993-014', '00767-003', '00276-010', '01516-013', '00089-011', '01553-003', '00078-016', '00209-013', '00788-015', '00651-010', '01556-014', '00677-004', '00335-015', '01580-010', '00120-014', '00948-007', '00595-015', '00361-003', '00861-004', '00963-015', '00028-014', '01501-014', '00735-016', '01225-006', '01673-015', '00745-006', '00736-004', '00550-018', '01101-002', '00091-010', '01662-016', '00611-009', '01114-002', '00531-016', '00490-009', '01622-012', '00780-013', '00120-003', '01303-003', '00420-014', '00302-012', '00403-006', '00642-013', '00473-008', '00194-009', '01069-016', '01280-005', '01534-008', '01100-016', '01630-002', '00188-008', '00256-017', '01211-013', '00368-006', '00686-002', '00796-004', '00034-005', '01071-014', '00749-009', '00681-011', '00617-004', '00760-008', '00040-010', '00424-008', '00683-009', '00406-012', '01212-014', '00569-013', '01321-002', '00068-002', '00415-005', '00064-006', '00913-005', '01445-008', '01264-006', '00130-004', '01471-009', '00477-007', '00915-002', '01615-003', '01268-006', '00125-002', '01560-007', '01553-016', '00093-014', '01164-013', '01586-016', '00373-017', '00906-010', '01632-013', '00454-015', '00883-014', '01263-017', '01391-008', '01301-010', '00196-015', '00986-014', '01516-011', '00883-003', '00260-014', '01548-012', '00774-003', '00346-012', '00933-008', '01668-003', '01303-006', '01442-014', '00241-008', '00972-001', '01594-015', '00868-013', '01199-005', '01026-006', '01590-008', '00328-010', '00683-017', '00647-001', '00548-012', '01598-014', '00311-005', '01192-015', '00821-019', '01555-016', '01284-001', '00950-005', '01063-014', '01597-014', '00074-010', '00054-018', '00573-002', '00302-009', '01282-017', '01566-007', '00146-016', '00095-018', '00490-016', '00752-017', '01517-012', '01493-014', '01076-006', '01361-008', '01296-004', '01082-018', '00491-014', '01220-003', '01018-016', '00320-004', '00118-006', '00562-002', '01362-009', '00919-007', '01410-013', '00572-004', '00205-004', '01508-005', '00440-015', '01168-002', '00032-009', '00775-003', '00642-005', '01181-003', '00263-015', '01375-003', '00573-017', '00950-017', '00414-017', '01043-013', '01113-002', '01026-005', '01687-008', '01033-001', '01224-009', '00618-010', '00409-001', '01220-007', '01313-012', '01369-014', '01585-017', '01112-016', '01362-004', '00639-012', '00525-010', '00704-008', '00255-005', '01263-015', '01088-001', '00722-001', '00928-014', '00932-017', '01200-010', '00681-015', '00077-015', '00091-009', '00812-017', '00313-012', '00078-003', '00455-013', '00128-005', '00200-002', '01677-002', '00415-011', '00225-002', '00277-016', '01642-009', '00062-016', '01681-008', '01125-016', '01258-017', '01557-016', '01382-001', '01169-016', '00308-016', '01078-006', '00086-013', '00627-015', '01558-014', '00130-002', '00906-006', '00950-004', '01255-003', '01217-013', '00480-015', '00837-018', '00386-015', '01263-003', '00098-003', '00412-017', '00977-012', '01120-006', '01188-006', '01199-011', '01402-011', '00481-010', '00956-015', '00815-004', '00095-012', '01129-015', '00923-016', '01430-017', '00031-006', '00930-007', '00156-010', '00799-017']
wrong_5 = ['00247-009', '00123-018', '00647-013', '01305-006', '01545-011', '01022-001', '01690-017', '00192-014', '01208-009', '00574-016', '00719-017', '01393-013', '00475-002', '00838-007', '00838-005', '01632-008', '00711-009', '01003-014', '00078-013', '00933-007', '00919-001', '00681-009', '01266-016', '01180-004', '00254-016', '00473-006', '00480-009', '00380-011', '01599-001', '01513-006', '00865-008', '01335-014', '00430-001', '00556-008', '00688-017', '00513-002', '00526-017', '00322-012', '01414-001', '00636-010', '00071-003', '00913-001', '01144-003', '01692-014', '01444-014', '01225-005', '00590-017', '00241-012', '01390-005', '00182-015', '00152-015', '00402-007', '00655-006', '01622-016', '00887-015', '00790-018', '00415-006', '00072-002', '00530-014', '00263-013', '00799-002', '00226-005', '00491-010', '00569-011', '01029-012', '01168-010', '00110-005', '00772-012', '00821-017', '00131-012', '00128-013', '01130-013', '01161-013', '00784-017', '01422-010', '00889-002', '01353-014', '01546-017', '00949-016', '01325-006', '01445-017', '00855-008', '01615-001', '01371-004', '00746-011', '00069-014', '00600-016', '00448-010', '00079-012', '01208-006', '00055-016', '00567-017', '01623-005', '01568-002', '00726-013', '00703-003', '00752-013', '00535-018', '00452-006', '00651-017', '00255-014', '01489-004', '01442-003', '00484-008', '01223-007', '00572-003', '01294-007', '00928-017', '01330-013', '00869-005', '01564-016', '00321-016', '00550-002', '01094-006', '01560-005', '01301-016', '01244-017', '00248-009', '01558-004', '00396-013', '00730-016', '00930-018', '01157-013', '01697-013', '00525-004', '01599-003', '00414-013', '00268-014', '01109-014', '00192-013', '01546-012', '00579-006', '01456-005', '00474-009', '00516-004', '01249-006', '00062-006', '00886-008', '00031-012', '01625-010', '01241-015', '01402-003', '00499-004', '00519-011', '01378-016', '01030-006', '00557-014', '01560-003', '00392-015', '00330-010', '00133-002', '00623-011', '01007-004', '01129-011', '00322-006', '01311-012', '00850-016', '00143-003', '00525-012', '00193-009', '00226-004', '00037-012', '00280-016', '01291-004', '01301-001', '00505-017', '00453-005', '00560-012', '00517-004', '00610-003', '00513-003', '00706-003', '00422-013', '00617-011', '00725-011', '00943-011', '00231-011', '00471-014', '00582-013', '01547-009', '01451-009', '01071-004', '01382-002', '01134-011', '00179-013', '01521-005', '01552-013', '00213-003', '01525-015', '01227-008', '00283-011', '00525-014', '01276-012', '00460-015', '00230-013', '00793-003', '01036-016', '01443-016', '00480-010', '00395-003', '00233-014', '00901-015', '01420-016', '01263-001', '01440-013', '00589-007', '00577-014', '01210-006', '00505-014', '01214-011', '01567-005', '01349-012', '00232-011', '00709-014', '00230-006', '00744-010', '00928-018', '00732-003', '01444-003', '01567-018', '00360-016', '00241-009', '00197-011', '00919-016', '01418-016', '00048-017', '00197-006', '00343-001', '00788-011', '01439-018', '01143-017', '01109-006', '01036-002', '00687-005', '00048-006', '00946-016', '01338-007', '00849-011', '01380-002', '00085-014', '01406-002', '00583-013', '01083-016', '01071-011', '01161-016', '00085-007', '00050-013', '01180-006', '00752-015', '01096-017', '01189-015', '00255-008', '00821-016', '00363-011', '01534-015', '00396-007', '01591-015', '00732-009', '00763-016', '00510-004', '00999-008', '00928-005', '01319-005', '00268-013', '01250-013', '00143-012', '01358-017', '00368-002', '01608-001', '01520-018', '01544-011', '01547-017', '00477-006', '01583-003', '00463-002', '01420-015', '00200-012', '00572-015', '00761-014', '00241-007', '00983-003', '00047-003', '00156-017', '01396-010', '00042-001', '01113-003', '00072-009', '00720-015', '00083-002', '01419-011', '00072-014', '01059-004', '00037-003', '01559-017', '00555-007', '00394-002', '01145-017', '00256-010', '01508-003', '00572-002', '01059-012', '01325-002', '00956-010', '00595-013', '00211-002', '01675-002', '00289-005', '00209-001', '00701-013', '00142-011', '00799-018', '00846-015', '00789-015', '01623-015', '01034-013', '01380-010', '01039-014', '00933-005', '01264-013', '01288-004', '00525-008', '01388-011', '01312-009', '00931-008', '00943-015', '01326-012', '01264-017', '00643-005', '00776-009', '01632-011', '00772-011', '00453-015', '01662-002', '00983-007', '00642-004', '00486-002', '00852-015', '00513-013', '01224-017', '01114-009', '00871-016', '01145-003', '00078-008', '00816-003', '00469-009', '01420-012', '00896-002', '00946-014', '00085-002', '00448-004', '00871-010', '01335-011', '00645-009', '01627-004', '00087-001', '00730-007', '00621-015', '00051-004', '01391-010', '00606-013', '01196-012', '01401-001', '00445-017', '00294-002', '01631-009', '00352-004', '01378-010', '00500-012', '01216-014', '01681-013', '00156-004', '01563-016', '00397-002', '01613-009', '00393-013', '01683-013', '01564-004', '00209-007', '01111-012', '00460-002', '00038-008', '01549-006', '00940-009', '00302-015', '00620-003', '01111-010', '00754-004', '01406-010', '00096-016', '00772-005', '00394-007', '01662-014', '01100-013', '01688-015', '01325-016', '01329-007', '00580-015', '00247-011', '01005-017', '00452-010', '00721-012', '01559-005', '00569-009', '00845-004', '00556-001', '00078-019', '00817-011', '00729-004', '00073-017', '00427-011', '00452-007', '01696-009', '00555-012', '00954-005', '00351-002', '00694-002', '00213-008', '01305-010', '01623-012', '00406-015', '01344-015', '01199-004', '00699-016', '00209-005', '01046-001', '01511-016', '01631-001', '00351-007', '01618-012', '00889-003', '00523-007', '00192-008', '00743-014', '00363-016', '00214-014', '00242-002', '00028-005', '01406-004', '01117-001', '00589-011', '01605-008', '01034-014', '01082-014', '00940-013', '01417-008', '00680-005', '00076-008', '01265-016', '00291-013', '00038-014', '00194-015', '00444-003', '00651-001', '00931-010', '01443-017', '01380-013', '00627-003', '00483-012', '00077-002', '01140-007', '01113-013', '01421-003', '00480-018', '01095-014', '01147-002', '00036-009', '01489-010', '01018-009', '00303-011', '01674-014', '01388-006', '00647-012', '00743-004', '00472-002', '01615-008', '01622-008', '01264-007', '00217-018', '00766-015', '01312-018', '00311-008', '00343-015', '01554-013', '00227-014', '00423-017', '00383-012', '00034-009', '01296-011', '01495-014', '01674-016', '00699-002', '00402-002', '01379-008', '01109-010', '00362-005', '00093-010', '00913-011', '01111-016', '00649-004', '00128-004', '00961-016', '01522-008', '01418-003', '00605-012', '00499-008', '00793-002', '01237-004', '01546-015', '00897-001', '00474-001', '00391-007', '00889-004', '00379-014', '00047-015', '01292-014', '01676-002', '00222-013', '00843-003', '01631-011', '01644-006', '00296-015', '00750-016', '00089-012', '00189-015', '00993-010', '00732-005', '00486-010', '00029-006', '01684-002', '00732-004', '01674-011', '01442-004', '00711-011', '00990-006', '00510-007', '00198-019', '01487-016', '00918-005', '00335-002', '01099-015', '01669-007', '01024-012', '00645-007', '01152-008', '01157-005', '01378-015', '01571-012', '00698-006', '01496-005', '00065-007', '01384-005', '01676-010', '01208-004', '01421-012', '00267-001', '00775-002', '00541-007', '00399-013', '01384-013', '01539-012', '00209-003', '01459-012', '00785-018', '00562-003', '00603-006', '00085-015', '01477-010', '01650-002', '00425-012', '01529-010', '01170-012', '00580-014', '00396-016', '00589-013', '00513-017', '00390-003', '00256-002', '00617-015', '00180-008', '01210-011', '01272-016', '00313-005', '01484-016', '00842-014', '00300-003', '01680-015', '01113-006', '00180-012', '01483-004', '01112-004', '00904-002', '00665-005', '00192-007', '00918-006', '00528-010', '01134-006', '00879-016', '00918-002', '00943-012', '00990-010', '00047-010', '01613-004', '00501-012', '00817-001', '00338-013', '01236-006', '00801-010', '01068-015', '01039-009', '01258-004', '01465-015', '01598-006', '00400-003', '01170-006', '01675-017', '01029-006', '01211-011', '00668-011', '01465-018', '01674-002', '00639-011', '01143-010', '01059-007', '01445-002', '00930-003', '01300-004', '00457-016', '00126-014', '01643-011', '01071-007', '01238-014', '00232-003', '01264-012', '01032-001', '00869-014', '01165-015', '00649-003', '00490-012', '00302-001', '00583-003', '00180-007', '01084-004', '01046-015', '01310-011', '01001-016', '01651-013', '00582-009', '00627-008', '01118-013', '00990-017', '00900-018', '01696-016', '01137-008', '00048-008', '00835-009', '00211-017', '00469-001', '00645-014', '00777-012', '00822-004', '01401-002', '00321-019', '01465-005', '00745-001', '00452-016', '00029-017', '00178-001', '00798-016', '01039-005', '01253-002', '00502-007', '01679-004', '00751-013', '00650-010', '00655-004', '01250-008', '01200-016', '00400-013', '00502-013', '00131-001', '00064-011', '01553-005', '01522-009', '00615-006', '01272-007', '01022-003', '01325-003', '01060-011', '00362-016', '00499-007', '00116-013', '00836-002', '01646-019', '01114-005', '01379-007', '01420-006', '01292-006', '00229-008', '00090-008', '00803-015', '00062-004', '00522-004', '01140-001', '01244-002', '00463-008', '00552-007', '00093-015', '00437-003', '00779-010', '00656-017', '00913-003', '00510-006', '00091-008', '00510-016', '00104-002', '00931-003', '00300-011', '00403-002', '00950-001', '00226-010', '00593-016', '01572-002', '00075-004', '00176-003', '01479-012', '01142-008', '00760-017', '01445-012', '01353-002', '00371-012', '00254-013', '01361-016', '00201-015', '01388-005', '00500-011', '01264-010', '01501-016', '00767-002', '00674-008', '00839-015', '00748-001', '01215-008', '01100-014', '00075-008', '00218-014', '00744-016', '01650-007', '00754-013', '01167-004', '00120-009', '00249-007', '01026-009', '00233-001', '00386-001', '00221-016', '00205-002', '00861-011', '00450-006', '00804-017', '01149-005', '01603-009', '00576-016', '00925-007', '01631-004', '00263-016', '00643-003', '00730-011', '01401-012', '00064-016', '00121-011', '01484-017', '01552-008', '01265-001', '01514-001', '00328-003', '01186-001', '00828-010', '00699-017', '00865-014', '00062-010', '00686-008', '01000-010', '00628-012', '00247-016', '01199-017', '01052-012', '01629-014', '01152-002', '01631-015', '00075-017', '01469-009', '00226-002', '01406-015', '01260-006', '00486-016', '01650-017', '00185-016', '01547-013', '01373-005', '00623-014', '00963-002', '01557-008', '00399-001', '01206-017', '00557-002', '01534-002', '01546-011', '00390-014', '00729-011', '00628-011', '00736-003', '01431-003', '01160-014', '00091-013', '00869-016', '01423-002', '00606-008', '00362-006', '00248-007', '00753-015', '00976-012', '00684-015', '00347-007', '01631-017', '01572-001', '00704-004', '01095-010', '00474-016', '00753-018', '01134-003', '01684-012', '00981-015', '00525-003', '01237-002', '01489-002', '00538-004', '01668-013', '00449-003', '01676-005', '00482-004', '01321-010', '00993-008', '01275-002', '01544-001', '00440-003', '01315-012', '01514-002', '00059-008', '00906-014', '01429-004', '01507-004', '01209-009', '01500-019', '01594-014', '00972-017', '01225-016', '00086-001', '00620-004', '00627-009', '01342-013', '00752-005', '01600-006', '00395-010', '01157-006', '00597-009', '00427-008', '00395-012', '00263-008', '00534-003', '00292-007', '01209-013', '00927-013', '01380-001', '01111-002', '00475-010', '00913-004', '00490-014', '00146-013', '00485-008', '00763-008', '00303-016', '01325-017', '01069-010', '00403-008', '00730-001', '00167-002', '00705-016', '00209-016', '00542-015', '00365-013', '00665-008', '00812-005', '01340-012', '01552-009', '01640-012', '00232-001', '00629-014', '01211-017', '00412-009', '01644-005', '00292-014', '01276-013', '00064-014', '00582-002', '01112-006', '00799-004', '00028-016', '00913-006', '01291-014', '01018-004', '01082-010', '01128-004', '01547-007', '00776-006', '00538-017', '01001-018', '01109-005', '00336-003', '01558-017', '00889-012', '01594-012', '00786-016', '00089-015', '01152-010', '01533-009', '00442-011', '01207-015', '00158-009', '01668-017', '00753-010', '01522-012', '00402-005', '01272-018', '01111-009', '00241-003', '01143-014', '00603-007', '00821-015', '00981-007', '00289-013', '01534-007', '00036-015', '00482-005', '01069-003', '00781-015', '01676-011', '00925-013', '00072-011', '00034-017', '01253-003', '00941-006', '01054-012', '00333-011', '00993-014', '00767-003', '00276-010', '01553-003', '00078-016', '00209-013', '00788-015', '00651-010', '00677-004', '00335-015', '01580-010', '00120-014', '00595-015', '00963-015', '00028-014', '01501-014', '00735-016', '01673-015', '00745-006', '00736-004', '00550-018', '01101-002', '00091-010', '01662-016', '00611-009', '01114-002', '00531-016', '00490-009', '01622-012', '00780-013', '00120-003', '01303-003', '00420-014', '00302-012', '00403-006', '00642-013', '00473-008', '00194-009', '01069-016', '01280-005', '01534-008', '01100-016', '01630-002', '00188-008', '00256-017', '01211-013', '00368-006', '00686-002', '00034-005', '00749-009', '00681-011', '00617-004', '00040-010', '00683-009', '00406-012', '00569-013', '01321-002', '00415-005', '00064-006', '01445-008', '01264-006', '00130-004', '01471-009', '00477-007', '00915-002', '01268-006', '00125-002', '01560-007', '00093-014', '01586-016', '00373-017', '00906-010', '01632-013', '00454-015', '00883-014', '01263-017', '01391-008', '01301-010', '00196-015', '00986-014', '01516-011', '00883-003', '00260-014', '01548-012', '00774-003', '00346-012', '00933-008', '01668-003', '01303-006', '01442-014', '00241-008', '00972-001', '01594-015', '00868-013', '01199-005', '01026-006', '01590-008', '00328-010', '00548-012', '01598-014', '00311-005', '01192-015', '00821-019', '01555-016', '01284-001', '00950-005', '01063-014', '01597-014', '00074-010', '00054-018', '00573-002', '00302-009', '01282-017', '00146-016', '00095-018', '00490-016', '00752-017', '01517-012', '01076-006', '01361-008', '01296-004', '01082-018', '00491-014', '01220-003', '01018-016', '00320-004', '00118-006', '00562-002', '01362-009', '00919-007', '01410-013', '00572-004', '00205-004', '01508-005', '00440-015', '01168-002', '00032-009', '00775-003', '00642-005', '01181-003', '00263-015', '01375-003', '00573-017', '00414-017', '01043-013', '01113-002', '01026-005', '01687-008', '01224-009', '00618-010', '00409-001', '01220-007', '01369-014', '01585-017', '01362-004', '00704-008', '01263-015', '00722-001', '00928-014', '00932-017', '00681-015', '00077-015', '00091-009', '00812-017', '00313-012', '00078-003', '00455-013', '00128-005', '00200-002', '01677-002', '00415-011', '00225-002', '00277-016', '01642-009', '00062-016', '01681-008', '01125-016', '01258-017', '01557-016', '01382-001', '01169-016', '00308-016', '01078-006', '00086-013', '00627-015', '01558-014', '00130-002', '00906-006', '00950-004', '01217-013', '00480-015', '00837-018', '00386-015', '01263-003', '00412-017', '00977-012', '01120-006', '01199-011', '00481-010', '00956-015', '00815-004', '00095-012', '01129-015', '00923-016', '01430-017', '00031-006', '00930-007', '00156-010', '00799-017']

common_items = set(wrong_1) & set(wrong_3) & set(wrong_5)

for row in test_dataset:
  if row["id"] in common_items:
    print(f"{row['id']}: {row['question']}")